In [1]:
#import packages
import numpy as np
import json
import scanpy as sc
from collections import OrderedDict
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import anndata as ad
import pickle

#To track memory
import psutil
import os

#Run command line args
import argparse


#spectra imports
import Spectra as spc
from Spectra import Spectra_util as spc_tl
from Spectra import K_est as kst
from Spectra import default_gene_sets


In [2]:
# #Pull the args
# parser = argparse.ArgumentParser(description='Run SPECTRA analysis with subsampled cells')
# parser.add_argument('--n_sample_cells', type=int, default=1000, 
#                     help='Number of cells to sample for SPECTRA analysis')

# # Parse the arguments
# args = parser.parse_args()

# Formatting Gene Set Dictionary

### Frankensteining SPECTRA's provided dictionary: We used this dictionary to generate the results in the paper: https://doi.org/10.1101/2022.12.20.521311

### Options for custom gene_set: We supply the Cytopus knowledge base to construct custom input gene set dictionaries for Spectra. For a tutorial visit the github repository: https://github.com/wallet-maker/cytopus


In [3]:
#Check memory usage
process = psutil.Process(os.getpid())
print(f"Memory usage: {process.memory_info().rss / 1024**3:.2f} GB")

Memory usage: 0.62 GB


In [4]:
#Load skin dataset

skin_data = sc.read_h5ad('/krummellab/data1/DSCoLab/AUTOIPI/papers/skin/data/DB3_54/harmony/sobj_01-13-25.h5ad')
skin_data

AnnData object with n_obs × n_vars = 261290 × 28683
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'doubletFinderScore', 'nCount_ADT', 'nFeature_ADT', 'percent.mt', 'percent.ribo', 'isotype_ctl_max', 'S.Score', 'G2M.Score', 'Phase', 'pool', 'batch', 'biospecimen', 'individual', 'subtype', 'disease_group', 'years_since_diagnosis', 'Age', 'Sex at Birth', 'sample_type', 'd_sample_type', 'location', 'cluster_label'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'
    obsm: 'X_harmony', 'X_pca', 'X_umap'
    varm: 'HARMONY', 'PCs'

In [13]:
#Add the SPECTRA mapping Emily created
mapping_df = pd.read_csv('~/gabi_rotation/skin_immune_mapping.csv')
spectra_mapping = dict(zip(mapping_df['cell_id'], mapping_df['spectra_annot']))

skin_data.obs['spectra_annotation'] = skin_data.obs.index.map(spectra_mapping)

skin_data.obs['combined_annotation'] = skin_data.obs['spectra_annotation'].fillna(skin_data.obs['cluster_label'])

skin_cats = set(skin_data.obs['combined_annotation'])

In [14]:
skin_cats

{'B_memory',
 'CD4_T',
 'CD8_T',
 'DC',
 'Endothelial Cells',
 'Fibroblasts',
 'Keratinocytes',
 'MDC',
 'Melanocytes',
 'NK',
 'Schwann Cells',
 'Smooth Muscle Cells',
 'T',
 'Treg',
 'pDC'}

In [16]:
!pwd

/c4/home/sanjayr


In [32]:
# load the default gene set dictionary from the Spectra paper:
jpath = '/c4/home/sanjayr/gabi_rotation/Spectra_dict.json'
f = open(jpath, 'r')
annotations = json.loads(f.read())

#Create a new nested dictionary to exactly match our gene set
skin_gene_set = {}

# Add existing valid keys and their values
for key in annotations:
    if key in skin_cats:
        skin_gene_set[key] = annotations[key]

# Add missing keys with empty dictionaries
for key in skin_cats:
    if key not in skin_gene_set:
        skin_gene_set[key] = {}

skin_gene_set['global'] = annotations['global']

In [35]:
for key in skin_gene_set.keys():
    print(f'key: {key}\n Length: {len(skin_gene_set[key])}')

sum([len(value) for key, value in skin_gene_set.items()])

key: B_memory
 Length: 4
key: CD4_T
 Length: 9
key: CD8_T
 Length: 7
key: DC
 Length: 3
key: MDC
 Length: 7
key: NK
 Length: 5
key: Treg
 Length: 1
key: pDC
 Length: 3
key: Fibroblasts
 Length: 7
key: Endothelial Cells
 Length: 2
key: Melanocytes
 Length: 0
key: Schwann Cells
 Length: 0
key: Smooth Muscle Cells
 Length: 0
key: T
 Length: 0
key: Keratinocytes
 Length: 0
key: global
 Length: 150


198

In [21]:
skin_gene_set = spc_tl.check_gene_set_dictionary(
    skin_data,
    skin_gene_set,
    obs_key='combined_annotation',
    global_key='global'
)

Cell type labels in gene set annotation dictionary and AnnData object are identical
Your gene set annotation dictionary is now correctly formatted.


In [10]:
#Check memory usage
process = psutil.Process(os.getpid())
print(f"Memory usage: {process.memory_info().rss / 1024**3:.2f} GB")

Memory usage: 10.20 GB


# Create var for highly_variable genes

In [11]:
#Create boolean mask for top 3000 highly variable genes

n_variable_genes = 3000

most_var_genes = set(skin_data.var['vst.variance'].sort_values(ascending=False)[:n_variable_genes].index)
skin_data.var['highly_variable'] = skin_data.var.index.isin(most_var_genes)

# Fit SPECTRA model (scaled down)

In [12]:
#Subsample the skin_data cells

n_sample_cells = 1000
skin_data_sample = sc.pp.subsample(skin_data, n_obs=n_sample_cells, copy=True, random_state=42)

skin_gene_set = spc_tl.check_gene_set_dictionary(
    skin_data_sample,
    skin_gene_set,
    obs_key='combined_annotation',
    global_key='global'
)

Cell type labels in gene set annotation dictionary and AnnData object are identical
Your gene set annotation dictionary is now correctly formatted.


In [13]:
eigen_gene_factors = None
with open('gabi_rotation/factor_estimate.pkl', 'rb') as f:
    eigen_gene_factors = pickle.load(f)
        
n_epochs = 2

model = spc.est_spectra(adata=skin_data_sample,
    gene_set_dictionary=skin_gene_set,
    use_highly_variable=True,
    cell_type_key="combined_annotation",
    L = eigen_gene_factors,
    use_weights=True,
    lam=0.1, # varies depending on data and gene sets, try between 0.5 and 0.001
    delta=0.001,
    kappa=None,
    rho=0.001,
    use_cell_types=True,
    n_top_vals=50,
    label_factors=True,
    overlap_threshold=0.2,
    clean_gs = True,
    min_gs_num = 3,
    num_epochs=n_epochs #here running only 2 epochs for time reasons, we recommend 10,000 epochs for most datasets
)

Cell type labels in gene set annotation dictionary and AnnData object are identical
Your gene set annotation dictionary is now correctly formatted.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:46<00:00, 23.05s/it]


In [23]:
skin_data_sample.obsm['SPECTRA_cell_scores']

array([[4.20382391e-05, 1.29154437e-06, 8.53013659e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.21752870e-06, 1.28752579e-06, 2.76567002e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.46070085e-06, 1.90925682e-06, 1.64054960e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.75627428e-04, 5.87940113e-06, 8.52831227e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.38056593e-05, 3.30769064e-06, 3.45998157e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.19536061e-06, 8.07745161e-06, 1.48886415e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [42]:
#Check memory usage
process = psutil.Process(os.getpid())
print(f"Memory usage: {process.memory_info().rss / 1024**3:.2f} GB")

Memory usage: 31.14 GB


In [44]:
#save the model uncompressed format
with open(f'gabi_rotation/models/skin_model_{n_sample_cells}.pickle', 'wb') as f:
    pickle.dump(model, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
#save the adtaa uncompressed format
with open(f'gabi_rotation/models/skin_adata_{n_sample_cells}.pickle', 'wb') as f:
    pickle.dump(skin_data, f, pickle.HIGHEST_PROTOCOL)

# LOAD SPECTRA Model

In [ ]:
#and load it like this:
with open(f'gabi_rotation/models/skin_model_{n_sample_cells}.pickle', 'rb') as f:
    model = pickle.load(f)